# 15. 클로저가 변수 스코프와 상호작용하는 법을 알자

* 클로저(closure) : 자신이 정의된 스코프에 있는 변수를 참조하는 함수
* 스코프(scope)   : 유효범위

#### 숫자리스트를 정렬할 때 특정 그룹의 숫자들이 먼저 오도록 우선순위 매기는 패턴
* 사용자 인터페이스 표현
* 다른 것 보다 중요한 메시지나 예외 이벤트를 먼저 보여줘야 할 때 유용

In [3]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0,x)
        return(1,x)
    values.sort(key=helper)

In [4]:
numbers = [8,3,1,2,5,4,7,6]
group = {2,3,5,7}

sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


1. 파이썬은 클로저를 지원한다. 이때문에 help함수가 sort_priority group인수에 접근할 수 있다.
2. 함수는 파이썬의 일급객체(first-class object)다. 이 말은 함수를 직접 참조하고, 별수에 할당하고, 다른 함수의 인수로 전달하고, 표현식과 if문 등 에서 비교할 수 있다는 의미다. 따라서 sort 메서드에서 클로저 함수를 key인수로 받을 수 있다.
3. 파이썬에는 튜플을 비교하는 특정한 규칙이 있다. 먼저 인덱스 0으로 아이템을 비교하고 다음에 인덱스1, 다음은 인덱스2와 같이 진행한다. helper 클로저의 반환 값이 정렬 순서를 분리된 두 그룹으로 나뉘게 한건 이규칙이다.

### 좀더 명확해 보이는 방식으로 수정

In [6]:
def sort_priority2(numbers, group):
    found = False          # 스코프 : 'sort_priority2
    def helper(x):
        if x in group:
            found = True   # 스코프 : helper  -> 안 좋음
            return (0,x)
        return(1,x)
    numbers.sort(key=helper)
    return found

In [7]:
found = sort_priority2(numbers, group)
print('Found :',found)
print(numbers)

Found : False
[2, 3, 5, 7, 1, 4, 6, 8]


#### 스코프 탐색 순서
1. 현재 함수의 스코프
2. (현재스코를 담고 있는 다른 함수같은) 감싸고 있는 스코프
3. 코드를 포함하고 있는 모듈의 스코프(전역 스코프)
4. (len이나 str같은 함수를 담고 있는) 내장 스코프

### 데이터 얻어오기
* 파이썬에는 클로저에서 데이터를 얻어오는 특별한 문법이 있다 : nonlocal문
* nonlocal : 특정 변수 이름에 할당할 때 스코프 탐색이 일어나야 함을 나타낸다.
* 제약 : 전역 변수의 오염을 피할 수는 있으나, 모듈 수준의 스코프까지 탐색은 불가능

In [10]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0,x)
        return (1,x)
    numbers.sort(key=helper)
    return found

* nonlocal문은 클로저에서 데이터를 다른 스코프에 할당하는 시점을 알아보기 쉽게 해 준다.
* nonlocal문은 변수 할당이 모듈 스코프에 직접 들어가게 하는 global문을 보완한다.
* 주의 : 간단한 함수 외에 쓰면 부작용을 알아내기가 함들다.

### nonlocal을 사용하기 복잡하다 -> 헬퍼클래스로 상태를 감싸는 방법

In [11]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0,x)
        return(1,0)

sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True

## 핵심정리
* 클로저 함수는 자신이 정의된 스코프 중 어디에 있는 변수도 참조할 수 있다.
* 기본적으로 클로저에서 변수를 할당하면 바깥쪽 스코프에는 영향을 미치지 않는다.
* 파이썬3에서는 nonlocal문을 사용하여 클로저를 감싸고 있는 스코프의 변수를 수정할 수 있음을 알린다.
* 간단한 함수 이외에는 nonlocal문을 사용하지 말자